Play with memory

In [12]:
#r "nuget: Microsoft.SemanticKernel, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.4.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.MongoDB, 1.4.0-alpha"
#r "nuget: Microsoft.Extensions.DependencyInjection"
#r "nuget: Microsoft.Extensions.Logging"
#r "nuget: Microsoft.Extensions.Logging.Console"
#r "nuget: Microsoft.Extensions.Logging.Debug"
#r "nuget: System.Linq.Async, 6.0.1"

#!import ../config/Settings.cs

using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;

using Kernel = Microsoft.SemanticKernel.Kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
var builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey);
builder.Services.AddLogging(c => c.AddConsole().AddDebug().SetMinimumLevel(LogLevel.Trace));                
builder.Build();

Installed Packages Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.Extensions.Logging, 8.0.0 Microsoft.Extensions.Logging.Console, 8.0.0 Microsoft.Extensions.Logging.Debug, 8.0.0 Microsoft.SemanticKernel, 1.4.0 Microsoft.SemanticKernel.Connectors.MongoDB, 1.4.0-alpha Microsoft.SemanticKernel.Plugins.Memory, 1.4.0-alpha System.Linq.Async, 6.0.1

Connect to MongoDb

Add *mongoDbConnectionString* property to secrets.json

In [13]:
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Connectors.MongoDB;

#pragma warning disable SKEXP0011, SKEXP0003, SKEXP0052, SKEXP0030
var memoryBuilder = new MemoryBuilder();
memoryBuilder.WithAzureOpenAITextEmbeddingGeneration(
        "mrtextembeddingada002",
        azureEndpoint, 
        apiKey)
    .WithMemoryStore(new MongoDBMemoryStore(Settings.MongoDbConnectionString, "sample_airbnb", "vector_index"));   
var memory = memoryBuilder.Build();

In [20]:
var response = await memory.SearchAsync("listingsAndReviews", "I want a duplex", limit: 1, minRelevanceScore: 0.4, withEmbeddings: true).FirstOrDefaultAsync();
Console.WriteLine(response);

Define/load two plugins to facilitate use of chat history

In [15]:
using Microsoft.SemanticKernel.Plugins.Memory;

var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            .Build();
kernel.Plugins.Clear();

#pragma warning disable SKEXP0003, SKEXP0052, SKEXP0030
// TextMemoryPlugin provides the "recall" function
kernel.ImportPluginFromObject(new TextMemoryPlugin(memory));

const string skPrompt = @"
Answer questions about available apartments or houses. 

Information about me, based on your prompt:
- {{recall $userInput}}

Chat:
{{$history}}
User: {{$userInput}}
ChatBot: ";

var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, new OpenAIPromptExecutionSettings { MaxTokens = 200, Temperature = 0.8 });

Initialize kernel arguments

In [16]:
#pragma warning disable SKEXP0052

var arguments = new KernelArguments();

#pragma warning disable SKEXP0003
arguments[TextMemoryPlugin.CollectionParam] = "listingsAndReviews";
arguments[TextMemoryPlugin.LimitParam] = "2";
arguments[TextMemoryPlugin.RelevanceParam] = "0.8";

Define chat loop

In [17]:
var history = "";
arguments["history"] = history;
Func<string, Task> Chat = async (string input) => {
    // Save new message in the kernel arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nChatBot: {answer}\n";

    history += result;
    arguments["history"] = history;
    
    // Show the bot response
    Console.WriteLine(result);
};

In [18]:
await Chat("Any apartments with 3 bedrooms?");


User: Any apartments with 3 bedrooms?
ChatBot: Yes, there are several apartments with 3 bedrooms available. Could you please provide me with more details such as the location you are interested in?



In [19]:
await Chat("I am looking for a duplex with three bedrooms in Rebeira");


User: I am looking for a duplex with three bedrooms in Rebeira
ChatBot: I'm sorry, but I couldn't find any available duplexes with three bedrooms in Rebeira. Is there anything else I can assist you with?



In [23]:
await Chat("Who wrote War and Peace?");


User: What is current job?
ChatBot: I currently work as a tourist operator.



In [24]:
await Chat("How many pages does it have?");


User: What is the population of the city where you live now?
ChatBot: The population of Seattle is approximately 753,675 as of 2021.



## Adding external data

In [29]:
const string memoryCollectionName = "SKGitHub";

var githubFiles = new Dictionary<string, string>()
{
    ["https://github.com/microsoft/semantic-kernel/blob/main/README.md"]
        = "README: Installation, getting started, and how to contribute",
    ["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/02-running-prompts-from-file.ipynb"]
        = "Jupyter notebook describing how to pass prompts from a file to a semantic plugin or function",
    ["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/00-getting-started.ipynb"]
        = "Jupyter notebook describing how to get started with the Semantic Kernel",
    ["https://github.com/microsoft/semantic-kernel/tree/main/samples/plugins/ChatPlugin/ChatGPT"]
        = "Sample demonstrating how to create a chat plugin interfacing with ChatGPT",
    ["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/src/Plugins/Plugins.Memory/VolatileMemoryStore.cs"]
        = "C# class that defines a volatile embedding store",
};

In [33]:
// Memory functionality is experimental
#pragma warning disable SKEXP0003, SKEXP0011, SKEXP0052
var memoryBuilder = new MemoryBuilder();
memoryBuilder.WithAzureOpenAITextEmbeddingGeneration(
    "mrtextembeddingada002",
    azureEndpoint, 
    apiKey);
memoryBuilder.WithMemoryStore(new VolatileMemoryStore());
var memory = memoryBuilder.Build();

In [34]:
Console.WriteLine("Adding some GitHub file URLs and their descriptions to a volatile Semantic Memory.");
var i = 0;
foreach (var entry in githubFiles)
{
    await memory.SaveReferenceAsync(
        collection: memoryCollectionName,
        description: entry.Value,
        text: entry.Value,
        externalId: entry.Key,
        externalSourceName: "GitHub"
    );
    Console.WriteLine($"  URL {++i} saved");
}

Adding some GitHub file URLs and their descriptions to a volatile Semantic Memory.
  URL 1 saved
  URL 2 saved
  URL 3 saved
  URL 4 saved
  URL 5 saved


In [35]:
string ask = "I love Jupyter notebooks, how should I get started?";
Console.WriteLine("===========================\n" +
                    "Query: " + ask + "\n");

var memories = memory.SearchAsync(memoryCollectionName, ask, limit: 5, minRelevanceScore: 0.77);

i = 0;
await foreach (var memory in memories)
{
    Console.WriteLine($"Result {++i}:");
    Console.WriteLine("  URL:     : " + memory.Metadata.Id);
    Console.WriteLine("  Title    : " + memory.Metadata.Description);
    Console.WriteLine("  Relevance: " + memory.Relevance);
    Console.WriteLine();
}

Query: I love Jupyter notebooks, how should I get started?

Result 1:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/00-getting-started.ipynb
  Title    : Jupyter notebook describing how to get started with the Semantic Kernel
  Relevance: 0.867688000202179

Result 2:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/README.md
  Title    : README: Installation, getting started, and how to contribute
  Relevance: 0.8083549737930298

Result 3:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/02-running-prompts-from-file.ipynb
  Title    : Jupyter notebook describing how to pass prompts from a file to a semantic plugin or function
  Relevance: 0.802147388458252

